In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib 

import colors, plot_func

In [ ]:
# k_stiff = np.array([1, 1])
# k_soft = np.array([1, 0.0001])
# thresh = np.array([40, 20])
# theta_ss = np.array([20, 20])
# init_buckle = np.array([1, 1])
# desired_buckle = np.array([-1, 1])

# k_stiff = np.array([4.2, 2.0, 1.0, 1.0])
# k_soft = np.array([0.2, 0.5, 0.1, 0.4])
# thresh = np.array([30, 25, 15, 10])
# theta_ss = np.array([25, 15, 10, 5])
# init_buckle = np.array([-1, 1, -1, -1])
# desired_buckle = np.array([1, -1, -1, 1])

k_stiff = np.array([4.2, 2.0, 1.0])
k_soft = np.array([0.2, 0.5, 0.1])
thresh = np.array([30, 25, 15])
theta_ss = np.array([5, 15, 10])
init_buckle = np.array([-1, -1, 1])
desired_buckle = np.array([1, 1, -1])

supress_prints = False

In [ ]:
iterations = 50
alpha = 0.025

In [ ]:
## Classes

class VariablesClass:
    """
    Class with variables dictated by supervisor
    """
    def __init__(self, k_stiff, k_soft, thresh, theta_ss):
        self.k_stiff = k_stiff
        self.k_soft = k_soft
        self.thresh = thresh
        self.theta_ss = theta_ss
        self.N_springs = np.size(k_stiff)
        
        # correct for un-physical threshold to move shim
        self.thresh[self.thresh<self.theta_ss] = self.theta_ss[self.thresh<self.theta_ss]
        
        # soft always 
        self.k_soft[self.k_soft>self.k_stiff] = self.k_stiff[self.k_soft>self.k_stiff]
    
    def set_normalizations(self, k_stiff, k_soft, theta_ss):
        self.k_bar = np.mean([np.mean(k_stiff), np.mean(k_soft)])
        self.theta_bar = np.mean(theta_ss)

        
class SupervisorClass:
    """
    Class with variables dictated by supervisor
    """
    def __init__(self, desired_buckle, alpha):
        self.desired_buckle = desired_buckle
        self.input_in_t = []
        self.input_update_in_t = []
        self.loss_in_t = []
        self.loss_MSE_in_t = []
        self.alpha = alpha
        
        self.input_update = 0
        self.input_update_in_t.append(self.input_update)
        
    def init_dataset(self, iterations):
        self.iterations = iterations
        # self.theta_vec = np.array([-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50])
        self.theta_vec = np.random.uniform(-90, 90, iterations)
    
    def desired_tau(self):
        self.desired_tau_vec = np.zeros(np.size(self.theta_vec))
        for i, theta in enumerate(self.theta_vec):
            self.desired_tau_vec[i] = tau_hinge(theta, self.desired_buckle, Variabs)
            
    def set_theta(self, iteration):
        self.theta = self.theta_vec[iteration]
        self.desired_tau = self.desired_tau_vec[i]
        print('theta ', self.theta)
        
    def calc_loss(self, tau):
        self.loss = loss(self.desired_tau, tau)
        self.loss_MSE = loss_MSE(self.loss)
        self.loss_in_t.append(self.loss)
        self.loss_MSE_in_t.append(self.loss_MSE)
        if not supress_prints:
            print('desired tau ', self.desired_tau)
            print('loss ', self.loss)
            print('MSE loss ', self.loss_MSE)
        
    def calc_input_update(self, State, Supervisor, Variabs):
        delta_theta = input_update(State.tau, Supervisor.loss, Supervisor.theta, Variabs.k_bar, Variabs.theta_bar)
        input_update_nxt = copy.copy(self.input_update) + self.alpha * delta_theta
        self.input_update = input_update_nxt
        self.input_update_in_t.append(self.input_update)
        if not supress_prints:
            print('delta theta', delta_theta)
            print('input_update ', self.input_update)


class StateClass:
    """
    Class with state variables
    """
    def __init__(self, Variabs, Supervisor, buckle=None):
        # self.buckle_in_t = np.zeros([Supervisor.iterations, Variabs.N_springs])
        # self.tau_in_t = np.zeros(Supervisor.iterations)
        self.buckle_in_t = []
        self.tau_in_t = []
        
        if buckle is not None:
            self.buckle = buckle
        else:
            self.buckle = np.ones(Variabs.N_springs)
        if not supress_prints:
            print('buckle pattern ', self.buckle)
        self.buckle_in_t.append(self.buckle)

    def calc_tau(self, theta, Variabs):
        self.tau = tau_hinge(theta, self.buckle, Variabs) 
        if not supress_prints:
            print('tau ', self.tau)
            
    def evolve_material(self, Supervisor, Variabs):
        buckle_nxt = np.zeros(Variabs.N_springs)
        for i in range(Variabs.N_springs):
            if self.buckle[i] == 1 and Supervisor.input_update > Variabs.thresh[i]:  # buckle left
                buckle_nxt[i] = -1
            elif self.buckle[i] == -1 and Supervisor.input_update < -Variabs.thresh[i]:  # buckle right
                buckle_nxt[i] = 1
            else:
                buckle_nxt[i] = self.buckle[i]
        self.buckle = copy.copy(buckle_nxt)
        self.buckle_in_t.append(self.buckle)
        if not supress_prints:
            print('buckle pattern ', self.buckle)       

In [ ]:
def tau_tot():
    return (np.sum(tau_hinge(theta, buckle, Variabs)**(-1)))**(-1)


def tau_hinge(theta, buckle, Variabs):
    return np.sum(tau_k(theta, buckle, Variabs))
    

def tau_k(theta, buckle, Variabs):
    tau_k = np.zeros(Variabs.N_springs)
    for i in range(Variabs.N_springs):
        if buckle[i] == 1 and theta > -Variabs.theta_ss[i] or buckle[i] == -1 and theta < Variabs.theta_ss[i]:
            k = Variabs.k_stiff[i]
        else:
            k = Variabs.k_soft[i]
        tau_k[i] = -k * (theta - (-buckle[i]) * Variabs.theta_ss[i])
    return tau_k


def loss(desired_tau, tau):
    return desired_tau - tau


def loss_MSE(loss):
    return np.mean(loss**2)


def input_update(tau, loss, theta, k_bar, theta_bar):
    tau_pre = 1
    theta_pre = -1
    # return (tau/k_bar-theta)*(loss)/(k_bar*theta_bar)
    # return (theta/theta_bar)*(loss)/(k_bar*theta_bar)
    return (tau_pre*tau/k_bar+theta_pre*theta)*(loss)/(k_bar*theta_bar)


def measure_full_response(Variabs, buckle, length=100):
    theta_vec = np.linspace(-180, 180, length)
    tau_vec = np.zeros([length])
    for i, theta in enumerate(theta_vec):
        tau_vec[i] = tau_hinge(theta, buckle, Variabs)
    return theta_vec, tau_vec

In [ ]:
Variabs = VariablesClass(k_stiff, k_soft, thresh, theta_ss)
Variabs.set_normalizations(k_stiff, k_soft, theta_ss)

In [ ]:
Supervisor = SupervisorClass(desired_buckle, alpha)
Supervisor.init_dataset(iterations)
Supervisor.desired_tau()

In [ ]:
State = StateClass(Variabs, Supervisor, buckle=init_buckle)

## Loop

In [ ]:
theta_full, tau_full_init = measure_full_response(Variabs, State.buckle)
theta_full, tau_full_desired  = measure_full_response(Variabs, desired_buckle)

for i in range(iterations):
    
    ## Measure
    Supervisor.set_theta(i)
    State.calc_tau(Supervisor.theta, Variabs)
    
    ## Loss
    Supervisor.calc_loss(State.tau)
    
    ## Update
    Supervisor.calc_input_update(State, Supervisor, Variabs)
    State.calc_tau(Supervisor.input_update, Variabs)
    State.evolve_material(Supervisor, Variabs)
    
    theta_full, tau_full_final = measure_full_response(Variabs, State.buckle)
    plot_func.plot_response(theta_full, tau_full_init, tau_full_final, tau_full_desired, theta_range=[-90, 90], just_init=False)

Supervisor.loss_MSE_norm_in_t = Supervisor.loss_MSE_in_t / (Variabs.k_bar * Variabs.theta_bar)

In [ ]:
importlib.reload(plot_func)

plot_func.importants(State.buckle_in_t, Supervisor.desired_buckle, 
                     Supervisor.loss_MSE_norm_in_t, Supervisor.input_update_in_t)

plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=True)
plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=False)

In [ ]:
plot_response(theta_full, tau_full_init, tau_full_final, tau_full_desired, theta_range=[-90, 90], just_init=True)
